#  Data collection & Munging

In dit notebook zal the ruwe datasets omgezet worden naar makkelijk bewerkbare data in een Pandas dataframe.

#### De XML dataset
In de XML dataset staat wel ontwikkeling zich afspelen op het veld. Je kunt denken aan bal winst of een goal. Ook wordt er bijghouden waar de ontwikkeling plaats vind. Bijvoorbeeld L2 of R1 (Zie afbeelding).
<img src="./voetbalveld-indeling.png" width=300>

##### XML structuur
De XML is als volgt op gebouwd:
```XML
<ALL_INSTANCES>
    <instance>
        <ID>1</ID>
        <start>4.3110428970</start>
        <end>40.9711876731</end>
        <code>TEG</code>
        <label>
            <group>Naar zone</group>
            <text>M2 TEG</text>
        </label>
    </instance>
</ALL_INSTANCES>
```
Er komen in ```<ALL_INSTANCES>``` meerdere instances voor.

Om een xml intelezen hebben van de XML library ElementTree nodig. Hiermee kunnen we makkelijk alle data uit de XML halen. We hebben ook pandas nodig omdat we pandas willen gebruiken voor de analyse.

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

De eerste stap zal zijn om de XML dataset om te zetten naar een pandas dataframe. Hiervoor maken we een functie waarin we de root van de dataset kunnen meegeven. We maken er een functie van omdat we dit voor meerdere datasets willen doen. 
<br />
<br />
In de functie zullen door alle childeren van de root gaan (we zitten dan in ```<instance>```) en daarvan de subchilderen. voor elke subchild checken we of het een label tag is. Dit doen we omdat de label tag meer subchilderen heeft. Als dat niet zo is dan voegen we de tekst van de tag toe aan de record en als de tag naam niet in de header list zit voegen we die ook toe. Als de tag wel een label loopen we door de childeren van die label tag. Een instance tag kan een verschillende hoeveelheid label tags bevatten. Daarom moeten we meerdere headers toevoegen voor de label tags. 
<br />
<br />
Zoals eerder gezegd heeft niet elke instance een gelijke hoeveelheid labels. Bijvoorbeeld de instance met id twee heeft maar 1 label terwijl instance met id 1 een meer dan 10 labels heeft. We zullen dus null waarde krijgen. Dit is niet erg we zullen gewoon de null waarde invullen.  
<br />
<br />
Als dat allemaal gedaan is krijgen we een mooie pandas dataset waar we mee kunnen werken.

In [2]:
tree = ET.parse('datasets/new-xml/20160410 FC Utrecht - NEC-new.xml')

root = tree.getroot()

def xmlRootToDataframe(root):
    all_instances = root[1]
    all_records = []
    headers = []
    for i, child in enumerate(all_instances):
        current_label = 0
        record = []
        for subchild in child:
            label_name = ''
            if subchild.tag == 'label':
                subrecord = []
                label_name = subchild.tag+'.'+str(current_label)
                if label_name not in headers:
                        headers.append(label_name)
                for subsubchild in subchild:
                    subrecord.append(subsubchild.text)
                current_label += 1
                record.append(subrecord)
            else:
                record.append(subchild.text)
            if subchild.tag not in headers and subchild.tag not in 'label':
                headers.append(subchild.tag)

        all_records.append(record)

    return pd.DataFrame(all_records, columns=headers)

df = xmlRootToDataframe(root)
df.head()

,ID,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,...,label.62,label.63,label.64,label.65,label.66,label.67,label.68,label.69,label.70,label.71
0,1,4.3110428970,40.9711876731,TEG,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
1,2,6.2150178417,15.1347948447,M2 TEG,"[Naar zone, L2 TEG]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,3,6.3110428970,40.9711876731,Start Jaimy,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
3,4,6.3110428970,40.9711876731,Verdedigen,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]",...,None,None,None,None,None,None,None,None,None,None
4,5,8.2200000000,14.0400000000,AFTRAP WB,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [3]:
df[['ID', 'start', 'end', 'code', 'label.0']].dtypes

ID         object
start      object
end        object
code       object
label.0    object
dtype: object

De datatypes van alle kolommen zijn nu objecten. Dit is niet idiaal en zal later beperkingen opleveren. laten we start en end numeric maken. Ook willen we categorical data maken van de code kolom. We maken de id kolom ook gelijk de index.
<br />
<br />
We beginnen de index veranderen daarna zetten we de datatypes om.

In [4]:
df = df.set_index('ID')
df.index.name = None

df[['start','end']] = df[['start','end']].apply(pd.to_numeric)
df.code = df.code.astype('category')

print(df[['start','end', 'code', 'label.0']].dtypes)

start       float64
end         float64
code       category
label.0      object
dtype: object


In [5]:
df.head()

,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,label.6,...,label.62,label.63,label.64,label.65,label.66,label.67,label.68,label.69,label.70,label.71
1,4.311043,40.971188,TEG,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
2,6.215018,15.134795,M2 TEG,"[Naar zone, L2 TEG]",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,6.311043,40.971188,Start Jaimy,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
4,6.311043,40.971188,Verdedigen,"[Naar zone, M2 TEG]","[Naar zone, L2 TEG]","[Naar zone, M1 TEG]","[Storen NEC, Compact]","[Storen NEC, Druk]","[Naar zone, R1 TEG]","[Naar zone, R2 TEG]",...,None,None,None,None,None,None,None,None,None,None
5,8.220000,14.040000,AFTRAP WB,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Laten we kijken hoeveel doelpunten zijn gemaakt.

In [ ]:
df[df['code'].str.contains("Goal")].index.tolist()
df.iloc[:1182].tail(10)

,start,end,code,label.0,label.1,label.2,label.3,label.4,label.5,label.6,...,label.62,label.63,label.64,label.65,label.66,label.67,label.68,label.69,label.70,label.71
1173,2570.299024,2588.370434,L4 NEC,"[Naar speler, gAAAAABYrzOUOTRugGK4u8cjcl60I8vj...","[Van speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQj...","[Naar speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQ...","[Van speler, gAAAAABYrzOUOTRugGK4u8cjcl60I8vjD...","[Rendement, Voorzet]","[Naar zone, R4 NEC]",None,...,None,None,None,None,None,None,None,None,None,None
1174,2572.150922,2585.082846,gAAAAABYrzOUOTRugGK4u8cjcl60I8vjDVJstq7eg1TC3R...,"[Van speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQj...","[Naar speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQ...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1175,2575.538481,2595.538481,Voorzet NEC,"[Rendement, Voorzet]","[Naar zone, R4 NEC]","[Naar speler, b6a6aeaf67e640fe9eb72d888ba046f9]","[Van speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQj...","[Naar speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ...","[Van speler, b6a6aeaf67e640fe9eb72d888ba046f9]","[Naar speler, 9c22deb258fa4a55bb736b89a40004d5]",...,None,None,None,None,None,None,None,None,None,None
1176,2576.606908,2595.195186,gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQjqXIfnAuYWfxtF...,"[Van speler, gAAAAABYrzOUOTRugGK4u8cjcl60I8vjD...","[Rendement, Voorzet]","[Naar zone, R4 NEC]","[Naar speler, b6a6aeaf67e640fe9eb72d888ba046f9]",None,None,None,...,None,None,None,None,None,None,None,None,None,None
1177,2582.195186,2597.083098,b6a6aeaf67e640fe9eb72d888ba046f9,"[Van speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQj...","[Naar speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1178,2584.083098,2594.810854,gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ09fQgxq6iILsRd...,"[Van speler, b6a6aeaf67e640fe9eb72d888ba046f9]","[Naar speler, 9c22deb258fa4a55bb736b89a40004d5]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1179,2585.370434,2600.474685,R4 NEC,"[Naar speler, b6a6aeaf67e640fe9eb72d888ba046f9]","[Van speler, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQj...","[Naar speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ...","[Van speler, b6a6aeaf67e640fe9eb72d888ba046f9]","[Naar speler, 9c22deb258fa4a55bb736b89a40004d5]","[Van speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ0...","[Naar speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ...",...,None,None,None,None,None,None,None,None,None,None
1180,2588.178857,2601.442851,9c22deb258fa4a55bb736b89a40004d5,"[Van speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ0...","[Naar speler, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1181,2589.555315,2609.555315,Kans NEC,"[Betrokken, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQjq...","[Betrokken, gAAAAABYrzOUOTRugGK4u8cjcl60I8vjDV...","[Betrokken, gAAAAABYrzOViQ5bQbZkXvcGYeCP26pkyc...","[Betrokken, b6a6aeaf67e640fe9eb72d888ba046f9]","[Betrokken, gAAAAABYrzOUx70t7JKQR_R_qB_fAsAQ_k...","[Betrokken, feaf75270e9c46178b6b36507d9576da]","[Betrokken, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ09...",...,None,None,None,None,None,None,None,None,None,None
1182,2589.883302,2609.883302,Goal NEC,[Goal],"[Betrokken, gAAAAABYrzOUOTRugGK4u8cjcl60I8vjDV...","[Betrokken, gAAAAABYrzOUCFm6Bomi2VGE_iRXHdIQjq...","[Betrokken, gAAAAABYrzOU_m3cv0-Mq0cAMfKcJ6PZ09...","[Betrokken, gAAAAABYrzOViQ5bQbZkXvcGYeCP26pkyc...","[Betrokken, gAAAAABYrzOUx70t7JKQR_R_qB_fAsAQ_k...","[Betrokken, b6a6aeaf67e640fe9eb72d888ba046f9]",...,None,None,None,None,None,None,None,None,None,None
